In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [76]:
def generate_dataset(nodes, levels, total_supplies, total_demands, 
                     transp_costs, capacities, random_state=None):
    # number of nodes per level
    n_nodes = [nodes // levels] * (levels - 1) + \
              [nodes % levels + nodes // levels]
    assert sum(n_nodes) == nodes # partition
    if random_state:
        np.random.seed(random_state)
    # supplies
    supplies = []
    for item_supply in total_supplies:
        item_supplies = []
        for i in range(n_nodes[0]):
            supply = np.random.poisson(item_supply // n_nodes[0])
            if i == n_nodes[0] - 1:
                item_supplies.append(item_supply - sum(item_supplies))
            else:
                item_supplies.append(min(supply, item_supply - sum(item_supplies)))
        supplies.append(item_supplies)
        assert sum(item_supplies) == item_supply # partition
    
    demands = []
#     supplies = [np.random.poisson(supply // n_nodes[0], n_nodes[0]) \
#                 for supply in total_supplies]
#     demands = [np.random.poisson(demand // n_nodes[-1], n_nodes[-1]) \
#                 for demand in total_demands]
    # K, I, transp_cost, supply_demand, node_capacity
    return n_nodes, supplies, demands

In [77]:
generate_dataset(20, 5, [20,30], [30,20], None, None)

([4, 4, 4, 4, 4], [[5, 5, 5, 5], [6, 7, 6, 11]], [])

In [26]:
np.random.poisson(100, (3,3))

array([[ 88, 102,  95],
       [104,  99, 112],
       [ 81, 112, 109]])

In [20]:
mean_length = 59 // 5
np.random.randn(3,2) + mean_length

array([[ 0.89238316,  0.5758449 ],
       [-0.4546978 ,  0.96651393],
       [ 0.86214971,  1.38867573]])